In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

from dataset_dir import excel_dir,stata_dir

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.max_rows = None
pd.options.display.max_columns = None
r=pd.read_stata(stata_dir()+"Teamup_women_Dataset_Lagos.dta",convert_categoricals=False)
r=r[r['level2']==7]

In [4]:
data=r.copy()
cut_starting=list(data.columns[:26])
cut_ending=list(data.columns[-23:])
for i,j in enumerate(cut_ending):
    if(j=="instanceid"):
        del cut_ending[i]
    

for c in cut_starting:
    del data[c]
for c in cut_ending:
    del data[c]

In [5]:
df=data.copy()

Exploratory data analysis

In [10]:
df.shape

(760, 10470)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 0 to 2287
Columns: 10470 entries, resp_select to instanceid
dtypes: float64(1449), int8(8749), object(272)
memory usage: 16.3+ MB


Drop redundant features

In [6]:
red=[]
for i in df.columns:
    if(df[i].isnull().sum()==df.shape[0]):
        red.append(i)
for i in red:
    del df[i]

In [7]:
df.shape

(760, 9824)

fill missing values with special number

In [8]:
for i in df.columns:
    df[i]=df[i].fillna(-91)

In [9]:
df.shape

(760, 9824)

In [10]:
y=pd.read_excel(excel_dir()+"y.xlsx")

In [11]:
y_ids=y['Respondent ID']

In [13]:
a=df[df['resp_select'].isin(y_ids)]
b=df[~df['resp_select'].isin(y_ids)]

In [16]:
b.shape[0]+a.shape[0]

760

In [17]:
target=y[['Respondent ID','profile']]
target['resp_select']=y['Respondent ID']
del target['Respondent ID']

In [39]:
m=pd.merge(a,target,how='right',on='resp_select')

In [40]:
m.shape

(107, 9825)

declare vector and target variables

In [41]:
del m['res_name']
del m['ward']
del m['hh_name']
del m['q102']
del m['q106_cal']
del m['q211_strategy1']
del m['instanceid']
del m['resp_select']
del m['strat_lab1']

for i in m.columns:
    if(i[0]=="v"):
        del m[i]
for i in m.columns:
    if('strategy' in i):
        del m[i]
for i in m.columns:
    if('cal' in i):
        del m[i]
for i in m.columns:
    if('lab' in i):
        del m[i]
m=m.replace('',-91)
for i in m.columns:
    if(i!='profile'):
        if(m[i].dtype=='object'):
            del m[i]

for i in m.columns:
    if(i!='profile'):
        m[i]=m[i].astype(int)   

le = LabelEncoder()

m['profile'] = le.fit_transform(m['profile'])



In [42]:
X = m.iloc[:, :-1].values
y = m.iloc[:, -1].values

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [45]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [46]:
# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [47]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[3 3]
 [1 1]
 [3 3]
 [1 1]
 [1 1]
 [3 3]
 [3 3]
 [1 1]
 [1 1]
 [1 3]
 [3 3]
 [1 1]
 [0 0]
 [0 0]
 [3 3]
 [0 0]
 [1 1]
 [0 3]
 [3 3]
 [1 0]
 [3 3]
 [1 1]
 [1 1]
 [1 3]
 [1 1]
 [1 1]
 [1 2]]


In [48]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 3  1  0  0]
 [ 0 11  0  0]
 [ 0  1  0  0]
 [ 1  2  0  8]]


0.8148148148148148